In [17]:
!pip install mne
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.datasets.sleep_physionet.age import fetch_data

In [19]:
[x,y] = fetch_data(subjects=[0,1], recording=[1])

Using default location ~/mne_data for PHYSIONET_SLEEP...


In [20]:
#asd

In [21]:
def get_data(count):

    [alice_files] = fetch_data(subjects=[count], recording=[1])

    raw_train = mne.io.read_raw_edf(alice_files[0], stim_channel='marker',
                                    misc=['rectal'])
    annot_train = mne.read_annotations(alice_files[1])

    raw_train.set_annotations(annot_train, emit_warning=False)

    annotation_desc_2_event_id = {'Sleep stage W': 1,
                                  'Sleep stage 1': 2,
                                  'Sleep stage 2': 3,
                                  'Sleep stage 3': 4,
                                  'Sleep stage 4': 4,
                                  'Sleep stage R': 5}

    annot_train.crop(annot_train[1]['onset'] - 30 * 60,
                 annot_train[-2]['onset'] + 30 * 60)
    raw_train.set_annotations(annot_train, emit_warning=False)

    events_train, _ = mne.events_from_annotations(
        raw_train, event_id=annotation_desc_2_event_id, chunk_duration=30.)

# create a new event_id that unifies stages 3 and 4
    event_id = {'Sleep stage W': 1,
                'Sleep stage 1': 2,
                'Sleep stage 2': 3,
                'Sleep stage 3/4': 4,
                'Sleep stage R': 5}
    tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

    epochs_train = mne.Epochs(raw=raw_train, events=events_train,
                              event_id=event_id, tmin=0., tmax=tmax, baseline=None)

    x = epochs_train.get_data()[:,:2,:]
    y = epochs_train.events[:,2]
    return x,y
x,y=get_data(0)
x.shape

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /root/mne_data/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Loading data for 841 events and 3000 original time points ...
0 bad epochs dropped


(841, 2, 3000)

In [22]:
x1,y1=get_data(0)
x2,y2=get_data(1)
x3,y3=get_data(2)
x4,y4=get_data(3)
x5,y5=get_data(4)
x6,y6=get_data(5)
x7,y7=get_data(6)
x8,y8=get_data(7)
x9,y9=get_data(8)
x10,y10=get_data(9)
x11,y11=get_data(10)
x12,y12=get_data(11)
x13,y13=get_data(12)
x14,y14=get_data(13)
x15,y15=get_data(14)
x16,y16=get_data(15)
x17,y17=get_data(16)
x18,y18=get_data(17)
x19,y19=get_data(18)
x20,y20=get_data(19)
x21,y21=get_data(20)

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /root/mne_data/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Loading data for 841 events and 3000 original time points ...
0 bad epochs dropped
Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /root/mne_data/physionet-sleep-data/SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
1103 matching events foun

In [23]:
#features.shape
for i in range(21):
    globals()['x'+str(i+1)] = np.std(globals()['x'+str(i+1)], axis=2)

In [24]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=1, max_iter=1000)
#mlp.fit(x1,y1)
mlp.partial_fit(globals()['x'+str(1)], globals()['y'+str(1)],[1,2,3,4,5]) 
for i in range(1,16):
   mlp.partial_fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
for i in range(16,21):
    y_pred = mlp.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')



Accuracy score: 0.3916083916083916
[[  0   0 216   0   0]
 [  0   0  55   0   0]
 [  0   0 448   0   0]
 [  0   0 165   0   0]
 [  0   0 260   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.39      1.00      0.56       448
  Sleep stage R       0.00      0.00      0.00       165
  Sleep stage W       0.00      0.00      0.00       260

       accuracy                           0.39      1144
      macro avg       0.08      0.20      0.11      1144
   weighted avg       0.15      0.39      0.22      1144

--------------------------------------------------------
Accuracy score: 0.3273453093812375
[[  0   0 176   0   0]
 [  0   0  21   0   0]
 [  0   0 328   0   0]
 [  0   0 215   0   0]
 [  0   0 262   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [26]:
from sklearn.linear_model import PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=1000, random_state=20,tol=1e-3)
#mlp.fit(x1,y1)
pac.partial_fit(globals()['x'+str(1)], globals()['y'+str(1)],[1,2,3,4,5]) 
for i in range(1,16):
   pac.partial_fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 

In [27]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
for i in range(16,21):
    y_pred = pac.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score: 0.3916083916083916
[[  0   0 216   0   0]
 [  0   0  55   0   0]
 [  0   0 448   0   0]
 [  0   0 165   0   0]
 [  0   0 260   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.39      1.00      0.56       448
  Sleep stage R       0.00      0.00      0.00       165
  Sleep stage W       0.00      0.00      0.00       260

       accuracy                           0.39      1144
      macro avg       0.08      0.20      0.11      1144
   weighted avg       0.15      0.39      0.22      1144

--------------------------------------------------------
Accuracy score: 0.3273453093812375
[[  0   0 176   0   0]
 [  0   0  21   0   0]
 [  0   0 328   0   0]
 [  0   0 215   0   0]
 [  0   0 262   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [28]:
 from sklearn.linear_model import SGDClassifier
 sgd = SGDClassifier(max_iter=1000, random_state=20,tol=1e-3)
#mlp.fit(x1,y1)
sgd.partial_fit(globals()['x'+str(1)], globals()['y'+str(1)],[1,2,3,4,5]) 
for i in range(1,16):
   sgd.partial_fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = sgd.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')



Accuracy score: 0.3916083916083916
[[  0   0 216   0   0]
 [  0   0  55   0   0]
 [  0   0 448   0   0]
 [  0   0 165   0   0]
 [  0   0 260   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.39      1.00      0.56       448
  Sleep stage R       0.00      0.00      0.00       165
  Sleep stage W       0.00      0.00      0.00       260

       accuracy                           0.39      1144
      macro avg       0.08      0.20      0.11      1144
   weighted avg       0.15      0.39      0.22      1144

--------------------------------------------------------
Accuracy score: 0.3273453093812375
[[  0   0 176   0   0]
 [  0   0  21   0   0]
 [  0   0 328   0   0]
 [  0   0 215   0   0]
 [  0   0 262   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [29]:
from sklearn.neighbors import KNeighborsClassifier
#Create the Model
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')


Accuracy score: 0.5052447552447552
[[109  10  41  18  38]
 [  5   3  30   2  15]
 [ 40   6 255 111  36]
 [ 21   0  10 134   0]
 [ 60   0 118   5  77]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.46      0.50      0.48       216
Sleep stage 3/4       0.16      0.05      0.08        55
  Sleep stage 1       0.56      0.57      0.57       448
  Sleep stage R       0.50      0.81      0.62       165
  Sleep stage W       0.46      0.30      0.36       260

       accuracy                           0.51      1144
      macro avg       0.43      0.45      0.42      1144
   weighted avg       0.49      0.51      0.49      1144

--------------------------------------------------------
Accuracy score: 0.34331337325349304
[[ 48   8  59  31  30]
 [  9   1   9   1   1]
 [119   6 113  58  32]
 [ 95   0  16 104   0]
 [ 42  17 116   9  78]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.15      0.27      0.20       176
Sleep sta

In [30]:
#AdaBoost example with training and test set
# ====================================================================== #Import AdaBoost ensemble model
from sklearn.ensemble import AdaBoostClassifier
#Import Tree model as a base classifier
from sklearn import tree
####Create an AdaBoost Ensemble Classifier

clf=AdaBoostClassifier(tree.DecisionTreeClassifier(),n_estimators=100,algorithm='SAMME', random_state=None,learning_rate=0.8)
#Train the model using the training sets 
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.5
[[105   2  79  17  13]
 [  8   1  42   1   3]
 [ 37   6 254 123  28]
 [ 25   0   3 137   0]
 [ 50   0 127   8  75]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.47      0.49      0.48       216
Sleep stage 3/4       0.11      0.02      0.03        55
  Sleep stage 1       0.50      0.57      0.53       448
  Sleep stage R       0.48      0.83      0.61       165
  Sleep stage W       0.63      0.29      0.40       260

       accuracy                           0.50      1144
      macro avg       0.44      0.44      0.41      1144
   weighted avg       0.50      0.50      0.48      1144

--------------------------------------------------------
Accuracy score: 0.2754491017964072
[[ 41   5  78  42  10]
 [ 11   0  10   0   0]
 [126   7 115  56  24]
 [116   0  12  87   0]
 [ 48   6 170   5  33]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.12      0.23      0.16       176
Sleep stage 3/4       0.0

In [31]:
#Gradient boosting example with training and test set
 #Import Gradient Boosting ensemble model
from sklearn.ensemble import GradientBoostingClassifier

#Create the Model
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=4)
#Train the model using the training sets 
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.4562937062937063
[[ 90   0  70  15  41]
 [  5   0  43   2   5]
 [ 12   0 269 149  18]
 [ 20   0  10 134   1]
 [ 26   0 199   6  29]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.59      0.42      0.49       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.46      0.60      0.52       448
  Sleep stage R       0.44      0.81      0.57       165
  Sleep stage W       0.31      0.11      0.16       260

       accuracy                           0.46      1144
      macro avg       0.36      0.39      0.35      1144
   weighted avg       0.42      0.46      0.41      1144

--------------------------------------------------------
Accuracy score: 0.3203592814371258
[[ 30   0  74  50  22]
 [  9   0   9   1   2]
 [ 93   0  76  97  62]
 [ 61   0   3 150   1]
 [ 38   0 153   6  65]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.13      0.17      0.15       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [32]:
#Import LDA model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
#Create a LDA Classifier
clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None)
#Train the model using the training sets 
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.43006993006993005
[[ 62   0  74  69  11]
 [  5   0  42   6   2]
 [ 17   0 265 166   0]
 [  0   0   0 165   0]
 [ 25   0 206  29   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.57      0.29      0.38       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.45      0.59      0.51       448
  Sleep stage R       0.38      1.00      0.55       165
  Sleep stage W       0.00      0.00      0.00       260

       accuracy                           0.43      1144
      macro avg       0.28      0.38      0.29      1144
   weighted avg       0.34      0.43      0.35      1144

--------------------------------------------------------
Accuracy score: 0.3413173652694611
[[ 43   0  76  52   5]
 [  8   0  13   0   0]
 [141   0 173  14   0]
 [ 99   0   9 107   0]
 [ 71   0 168   4  19]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.12      0.24      0.16       176
Sleep sta

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [33]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
clf = GaussianNB()
#Train the model using the training sets
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.458041958041958
[[ 91   0  87  22  16]
 [  5   0  34   4  12]
 [ 19   0 247 175   7]
 [  6   0   0 159   0]
 [ 32   0 190  11  27]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.59      0.42      0.49       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.44      0.55      0.49       448
  Sleep stage R       0.43      0.96      0.59       165
  Sleep stage W       0.44      0.10      0.17       260

       accuracy                           0.46      1144
      macro avg       0.38      0.41      0.35      1144
   weighted avg       0.45      0.46      0.41      1144

--------------------------------------------------------
Accuracy score: 0.4121756487025948
[[ 31   0  75  52  18]
 [  5   0  14   0   2]
 [100   0 190  38   0]
 [ 59   0   3 153   0]
 [ 17   0 202   4  39]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.15      0.18      0.16       176
Sleep stage

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [34]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf = QuadraticDiscriminantAnalysis(store_covariance=True) 
#Train the model using the training sets
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.5034965034965035
[[109   0  54  20  33]
 [ 10   0  35   1   9]
 [ 57   0 250 128  13]
 [ 17   0   0 148   0]
 [ 72   0 114   5  69]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.41      0.50      0.45       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.55      0.56      0.55       448
  Sleep stage R       0.49      0.90      0.63       165
  Sleep stage W       0.56      0.27      0.36       260

       accuracy                           0.50      1144
      macro avg       0.40      0.45      0.40      1144
   weighted avg       0.49      0.50      0.48      1144

--------------------------------------------------------
Accuracy score: 0.3003992015968064
[[ 56   0  62  43  15]
 [ 11   0   9   0   1]
 [166   0 112  35  15]
 [146   0   1  68   0]
 [ 57   0 137   3  65]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.13      0.32      0.18       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminan

In [37]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
clf=SVC(kernel='poly',C=1,degree=2,coef0=1.001 ,gamma='auto')
clf.fit(globals()['x'+str(1)], globals()['y'+str(1)]) 
for i in range(1,16):
   clf.fit(globals()['x'+str(i+1)], globals()['y'+str(i+1)]) 
for i in range(16,21):
    y_pred = clf.predict(globals()['x'+str(i+1)])

    acc = accuracy_score(globals()['y'+str(i+1)], y_pred)

    print("Accuracy score: {}".format(acc))
    print(confusion_matrix(globals()['y'+str(i+1)], y_pred))
    print(classification_report(globals()['y'+str(i+1)], y_pred, target_names={'Sleep stage W',
                    'Sleep stage 1',
                    'Sleep stage 2',
                    'Sleep stage 3/4',
                    'Sleep stage R'}))
    print('--------------------------------------------------------')

Accuracy score: 0.3916083916083916
[[  0   0 216   0   0]
 [  0   0  55   0   0]
 [  0   0 448   0   0]
 [  0   0 165   0   0]
 [  0   0 260   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       216
Sleep stage 3/4       0.00      0.00      0.00        55
  Sleep stage 1       0.39      1.00      0.56       448
  Sleep stage R       0.00      0.00      0.00       165
  Sleep stage W       0.00      0.00      0.00       260

       accuracy                           0.39      1144
      macro avg       0.08      0.20      0.11      1144
   weighted avg       0.15      0.39      0.22      1144

--------------------------------------------------------
Accuracy score: 0.3273453093812375
[[  0   0 176   0   0]
 [  0   0  21   0   0]
 [  0   0 328   0   0]
 [  0   0 215   0   0]
 [  0   0 262   0   0]]
                 precision    recall  f1-score   support

  Sleep stage 2       0.00      0.00      0.00       176
Sleep stag

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat